In [49]:
import os

import pandas as pd
from dotenv import load_dotenv
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, PointStruct, VectorParams
from sentence_transformers import SentenceTransformer

load_dotenv()

True

In [109]:
df_1 = pd.read_csv(os.path.join(os.getcwd(), "data", "комиксы.csv"), on_bad_lines="skip").dropna()
df_2 = pd.read_csv(os.path.join(os.getcwd(), "data", "средневековье.csv"), on_bad_lines="skip").dropna()
df_3 = pd.read_csv(os.path.join(os.getcwd(), "data", "художественная.csv"), on_bad_lines="skip").dropna()

df = pd.concat([df_1, df_2, df_3], axis=0)


In [110]:
import uuid

df["ID"] = [uuid.uuid5(uuid.NAMESPACE_DNS, title).hex for title in df["Title"]]


In [111]:
df.head()

,Title,Author,Link,Image,Category,Description,Info,ID
0,Истории книжных магазинов,Ивашкина М.,https://www.podpisnie.ru/books/istorii-knizhny...,/upload/resize_cache/iblock/cfe/160_230_1/8vg4...,Графические романы / Комиксы,Книжный магазин — место где книга и человек на...,Автор Ивашкина М. Издательство Миля Год издани...,e26498c9c4a75252a18a1c677b6d3ae9
1,Полуночники,Эвенс Б.,https://www.podpisnie.ru/books/polunochniki/,/upload/resize_cache/iblock/a5b/160_230_1/tj5v...,Графические романы / Комиксы,Одна ночь. Три незнакомца. Три совершенно разн...,Автор Эвенс Б. Издательство Бумкнига Год издан...,2d4a036599e959a0a9e76e32a2075200
2,Госпожа Кагуя: В любви как на войне. Любовная...,Акасака А.,https://www.podpisnie.ru/books/gospozha-kaguya...,/upload/resize_cache/iblock/7fd/160_230_1/uqfn...,Графические романы / Комиксы,Завещание Ганъана Синомии уничтожено и Кагуя в...,Автор Акасака А. Издательство Азбука Год издан...,9ac06e3bafd25975bbf6fb9083e18567
3,"Люди, которые легко становятся счастливыми. ...",Дэнсинг С.,https://www.podpisnie.ru/books/lyudi-kotorye-l...,/upload/resize_cache/iblock/929/160_230_1/y4yj...,Графические романы / Комиксы,Привычная рутина отбирает краски жизни а о «ма...,Автор Дэнсинг С. Издательство КоЛибри Год изда...,ca68d65336425ee49bb1e563fb832e1f
4,"Моя геройская академия. Кн. 19. Те, кто об...",Хорикоси К.,https://www.podpisnie.ru/books/moya-geroyskaya...,/upload/resize_cache/iblock/f6d/160_230_1/7hh3...,Графические романы / Комиксы,Решающее сражение за судьбу мира в самом разга...,Автор Хорикоси К. Издательство Азбука Год изда...,2cb378c590605ddb8c4cf85b8ee1518f


In [45]:
embedder = SentenceTransformer(os.getenv("QDRANT_EMBEDDER"))
qdrant = QdrantClient(url=os.getenv("QDRANT_URL"), api_key=os.getenv("QDRANT_API_KEY"))


In [48]:
vectors = embedder.encode(df["Description"].values, batch_size=32, normalize_embeddings=True, show_progress_bar=True)

Batches:   0%|          | 0/627 [00:00<?, ?it/s]

In [113]:
qdrant.recreate_collection(
    collection_name=os.getenv("QDRANT_COLLECTION_NAME"),
    vectors_config=VectorParams(size=312, distance=Distance.COSINE),
)


C:\Users\S\AppData\Local\Temp\ipykernel_17908\3353009504.py:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [114]:
from tqdm import tqdm

for uid, category, author, title, image_link, vector in tqdm(zip(
    df["ID"], df["Category"], df["Author"], df["Title"], df["Image"], vectors
)):
    qdrant.upsert(
        collection_name=os.getenv("QDRANT_COLLECTION_NAME"),
        points=[
            PointStruct(
                id=uid,
                payload={
                    "category": category,
                    "author": author,
                    "title": title,
                    "image_link": image_link,
                },
                vector=vector,
            )
        ],
    )


0it [00:00, ?it/s]

20055it [43:51,  7.62it/s]


In [117]:
query_vector = embedder.encode("sdfsdfsfdsf")
n_chunks = 2
offset = 0

In [122]:
message_content = qdrant.search(
        collection_name=os.getenv("QDRANT_COLLECTION_NAME"), query_vector=query_vector, limit=n_chunks, offset=offset
    )

In [123]:
message_content

[ScoredPoint(id='199fb4a3-e956-5e2b-ab3d-200bc7e99e4d', version=1093, score=0.83818316, payload={'author': 'Маргопулос Р.', 'category': 'Графические романы / Комиксы', 'image_link': '/upload/resize_cache/iblock/c5b/160_230_1/qjrm705ftl3lwoyi6c41a6128hp2i6z7.jpg', 'title': 'MARVEL: Что если? .  .  Росомаха убил Халка'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id='421833ad-e246-56dc-9a16-447a4d64a915', version=1605, score=0.8188624, payload={'author': 'Ли С.', 'category': 'Графические романы / Комиксы', 'image_link': '/upload/resize_cache/iblock/1e8/160_230_1/7x70g1v5p394xkbbm75dpk7wum6oemw7.jpg', 'title': 'Удивительное фэнтези #15.  Первое появление Человека-Паука'}, vector=None, shard_key=None, order_value=None)]